<a href="https://colab.research.google.com/github/Gblack98/PestVisionAI/blob/Gabar_001/Tomato_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
noulam_tomato_path = kagglehub.dataset_download('noulam/tomato')

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import random
import cv2  # Pour la lecture d'images si besoin (optionnel)
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Configuration centrale des hyperparamètres
IMG_SIZE = 128
BATCH_SIZE = 64
EPOCHS = 100
NUM_CLASSES = 10
SEED = 1337

# Reproductibilité
tf.keras.utils.set_random_seed(SEED)

# Fonction de création du modèle amélioré
def create_model(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    model = Sequential([
        # Bloc convolutif 1
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Bloc convolutif 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.35),

        # Bloc convolutif 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.45),

        # Partie entièrement connectée
        Flatten(),
        Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

# Préparation des données avec augmentation avancée
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Chargement des données
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/tomato/New Plant Diseases Dataset(Augmented)/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

val_generator = val_datagen.flow_from_directory(
    '/kaggle/input/tomato/New Plant Diseases Dataset(Augmented)/valid',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Création et compilation du modèle
model = create_model()
optimizer = Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks améliorés
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, min_lr=1e-7),
    ModelCheckpoint('best_model.keras', save_best_only=True)
]

# Entraînement
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
# Évaluation finale
final_loss, final_acc = model.evaluate(val_generator)
print(f'\nValidation Accuracy: {final_acc:.2%}')
print(f'Validation Loss: {final_loss:.4f}')


In [ ]:
# -------------------------------
# Visualisations avancées
# -------------------------------

# 1. Courbes d'entraînement (Loss et Accuracy)
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Loss entraînement')
plt.plot(history.history['val_loss'], label='Loss validation')
plt.title('Courbes de Loss')
plt.xlabel('Époque')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Accuracy entraînement')
plt.plot(history.history['val_accuracy'], label='Accuracy validation')
plt.title('Courbes d\'Accuracy')
plt.xlabel('Époque')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# 2. Matrice de confusion et rapport de classification
# Obtenir les prédictions sur l'ensemble de validation
val_generator.reset()
Y_pred = model.predict(val_generator, steps=val_generator.samples // BATCH_SIZE + 1)
y_pred = np.argmax(Y_pred, axis=1)
y_true = val_generator.classes

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)
class_labels = list(val_generator.class_indices.keys())

plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.title('Matrice de Confusion')
plt.xlabel('Prédiction')
plt.ylabel('Vérité')
plt.show()

# Rapport de classification
print("Rapport de Classification :")
print(classification_report(y_true, y_pred, target_names=class_labels))

In [ ]:
# 3. Affichage d'exemples d'images avec leurs prédictions
# Récupérer un batch d'images de validation
val_batch = next(val_generator)
images, labels = val_batch[0], val_batch[1]
predictions = model.predict(images)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(labels, axis=1)

plt.figure(figsize=(20,10))
for i in range(12):
    idx = random.randint(0, images.shape[0]-1)
    plt.subplot(3,4,i+1)
    plt.imshow(images[idx])
    plt.title(f"Vrai: {class_labels[true_labels[idx]]}\nPréd: {class_labels[predicted_labels[idx]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()